# 1. Extracción de datos
En esta sección se cargan los datos de Grammys desde MySQL y de Spotify desde el archivo CSV a DataFrames de pandas.

In [ ]:
import pandas as pd
import pymysql

# --- Configuración de conexión MySQL ---
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="raw",
)

# --- Cargar Grammys desde MySQL ---
grammys_df = pd.read_sql("SELECT * FROM grammys_raw", conn)
print(f"Grammys shape: {grammys_df.shape}")
display(grammys_df.head())

# --- Cargar Spotify desde CSV ---
spotify_df = pd.read_csv("data/spotify_dataset.csv")
print(f"Spotify shape: {spotify_df.shape}")
display(spotify_df.head())

conn.close()

C:\Users\fabia\AppData\Local\Temp\ipykernel_17360\1480248473.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  grammys_df = pd.read_sql("SELECT * FROM grammys_raw", conn)


Grammys shape: (4810, 10)


,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,1
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,1
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,1
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,1
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,1


Spotify shape: (114000, 21)


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [ ]:
# --- Eliminar columnas irrelevantes ---
# Spotify: conservar solo columnas clave
spotify_cols = [
    'track_id', 'track_name', 'artists', 'album_name', 'popularity', 'duration_ms', 'track_genre'
]
spotify_df = spotify_df[[c for c in spotify_cols if c in spotify_df.columns]]

# Grammys: conservar solo columnas clave
grammys_cols = ['year', 'category', 'artist', 'winner']
grammys_df = grammys_df[[c for c in grammys_cols if c in grammys_df.columns]]

# --- Gestionar nulos en columnas clave ---
spotify_df = spotify_df.dropna(subset=['track_id', 'track_name', 'artists'])
grammys_df = grammys_df.dropna(subset=['year', 'category', 'artist'])

# --- Normalizar nombres de artistas ---
import re, unicodedata

def normalize_artist(name):
    if pd.isna(name) or str(name).strip() == "":
        return ""
    s = str(name).lower()
    s = ''.join(ch for ch in unicodedata.normalize('NFKD', s) if not unicodedata.combining(ch))
    s = re.sub(r"\(.*?\)|\[.*?\]", "", s)
    s = re.split(r"\s+(?:feat\.|featuring|ft\.|feat|featuring:|feats)\s+", s)[0]
    s = s.replace('&', ' and ')
    s = re.sub(r"[^a-z0-9\s]", '', s)
    s = re.sub(r"^the\s+", '', s)
    s = re.sub(r"\s+", ' ', s).strip()
    return s

spotify_df['artist_norm'] = spotify_df['artists'].apply(lambda x: normalize_artist(str(x).split(';')[0]))
grammys_df['artist_norm'] = grammys_df['artist'].apply(normalize_artist)

# --- Normalizar categorías de premios (opcional) ---
def normalize_category(cat):
    if pd.isna(cat): return ""
    s = str(cat).lower().strip()
    s = re.sub(r"[^a-z0-9 ]", '', s)
    return s

grammys_df['category_norm'] = grammys_df['category'].apply(normalize_category)

# --- Mostrar resultados preliminares ---
print("Spotify columnas:", spotify_df.columns.tolist())
print("Grammys columnas:", grammys_df.columns.tolist())
print("Ejemplo artista normalizado Spotify:", spotify_df['artist_norm'].head(3).tolist())
print("Ejemplo artista normalizado Grammys:", grammys_df['artist_norm'].head(3).tolist())

Spotify columnas: ['track_id', 'track_name', 'artists', 'album_name', 'popularity', 'duration_ms', 'track_genre', 'artist_norm']
Grammys columnas: ['year', 'category', 'artist', 'winner', 'artist_norm', 'category_norm']
Ejemplo artista normalizado Spotify: ['gen hoshino', 'ben woodward', 'ingrid michaelson']
Ejemplo artista normalizado Grammys: ['billie eilish', 'bon iver', 'ariana grande']
